In [1]:
#@title Configuration File

#Configuration File

#Do not change, these are ipython notebook demonstration

#Path for data
dataPath = 'https://github.com/nick-dahl/AI_Finance/blob/main/Data2016.csv?raw=true'

#Define the factor names
#factorName = ['World Equities Excess Return','Treasury Bond Excess Return','Default Risk Premium','Inflation Protection','Currency Protection']
factorName = ['World Equities', '10-year US Treasuries', 'High Yield', 'Inflation Protection', 'Currency Protection']

#Names of assets
#assetName = ['US Equities Excess Return','Real Estate Excess Return','Commodities']
assetName = ['SP500 Total Return','International Equity','U.S. Treasury 20 years', 'Corporate Bond','Real Estate', 'Commodity', 'TIPS']

#Name of date column
dateName = 'Date'

#User Analysis Section.  Change the variables in this section to run user specific analysis

#isDemo is a boolean variable, set to True if the user wants to run custom analysis
isDemo = False

#dataPathUser: Path to User Defined Data
dataPathUser = 'https://github.com/nick-dahl/AI_Finance/blob/main/Data2016.csv?raw=true'

#factorNameUser: List, defines the factors
factorNameUser = ['World Equities', '10-year US Treasuries', 'High Yield', 'Inflation Protection', 'Currency Protection']

#assetNameUser: List, defines the asset to be used
assetNameUser = 'Commodity'

#dateName: string, date column
dateNameUser = 'Date'

#lambdaHatUser: float, penalty term of user define LASSO regression
lambdaHatUser = .00005

#Start and End Dates for the Analysis
startDateUser = '1997-03-01'
endDateUser = '2014-12-01'

#Stuff for optional part of user section

#Best Subset Regression Related
maxVarsUser = 1

#Elastic Net Related
numL1RatioUser = 10
numAlphasUser = 20


In [2]:
import cvxpy as cp
print(cp.__version__)


1.7.1


In [ ]:
#@title Library for Factor-Model usage

#Library for Factor-Model usage
import numpy as np #for numerical array data
import pandas as pd #for tabular data
from scipy.optimize import minimize
import matplotlib.pyplot as plt #for plotting purposes
import time

import cvxpy as cp
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#Plotting Functions
def plot_returns(data, names, flag='Total Return', date='Date', printFinalVals = False):
    '''plot_returns returns a plot of the returns
    INPUTS:
        names: string, name of column to be plotted, or list, in which case it plots all of them
        data: pd dataframe, where the data is housed
        flag: string, Either Total Return or Monthly Return
        date: string, column name corresponding to the date variable
        printFinalVals: Boolean, if True, prints the final Total Return
    Outputs:
        a plot'''
    #Clean Inputs:
    if(date not in data.columns):
        print ('date column not in the pandas df')
        return
    if(type(names) is str):
        names = [names]
    for name in names:
        if(name not in data.columns):
            print ('column ' + name + ' not in pandas df')
            return
    #If the inputs are clean, create the plot
    data = data.sort_values(date).copy()
    data.reset_index(drop=True, inplace=True)
    data[date] = pd.to_datetime(data[date])

    if (flag == 'Total Return'):
        n = data.shape[0]
        totalReturns = np.zeros((n,len(names)))
        totalReturns[0,:] = 1.
        for i in range(1,n):
            totalReturns[i,:] = np.multiply(totalReturns[i-1,:], (1+data[names].values[i,:]))
        for j in range(len(names)):
            plt.semilogy(data[date], totalReturns[:,j])

        plt.title('Total Return Over Time')
        plt.ylabel('Total Return')
        plt.legend(names)
        plt.xlabel('Date')
        plt.show()
        if(printFinalVals):
            print(totalReturns[-1])
    elif (flag == 'Return'):
        for i in range(len(names)):
            plt.plot(data[date], data[names[i]])
        plt.title('Returns Over Time')
        plt.ylabel('Returns')
        plt.legend(names)
        plt.xlabel('Date')
        plt.show()
    else:
        print ('flag variable must be either Total Return or Return')


#Helper Functions
def create_options():
    '''create standard options dictionary to be used as input to regression functions'''
    options = dict()
    options['time_period'] = 'all'
    options['date'] = 'Date'
    options['return_model'] = False
    options['print_loadings'] = True
    return options

def create_options_lasso():
    options = create_options()
    options['lambda_hat'] = .5
    return options

def create_options_ridge():
    options = create_options()
    options['lambda'] = 1
    return options

def create_options_cv_lasso():
    options = create_options()
    options['max_lambda_hat'] = 1
    options['n_lambda_hat'] = 200
    options['random_state'] = 7777
    options['n_folds'] = 10
    return options

def create_options_cv_ridge():
    options = create_options()
    options['max_lambda'] = 1
    options['n_lambda'] = 100
    options['random_state'] = 7777
    options['n_folds'] = 10
    return options

def create_options_cv_elastic_net():
    options = create_options()
    options['max_lambda_hat'] = 1
    options['max_l1_ratio'] = .99
    options['n_lambda_hat'] = 100
    options['n_l1_ratio'] = 20
    options['random_state'] = 7777
    options['n_folds'] = 10
    return options

def create_options_best_subset():
    '''create standard options dictionary to be used as input to regression functions'''
    options = create_options()
    options['return_model'] = False
    options['print_loadings'] = True
    options['max_vars'] = 3
    return options


def create_dictionary_for_analysis(method, methodDict=None):
    '''create_dictionary_for_anlsis creates the options dictionary that can be used as an input to a factor model
    INPUTS:
        method: string, defines the method
    OUTPUTS:
        methodDict: dictionary, keys are specific options the user wants to specify, values are the values of those options
    '''
    if(method == 'OLS'):
        options = create_options()
    elif(method == 'CVLasso'):
        options = create_options_cv_lasso()
    elif(method == 'CVRidge'):
        options = create_options_cv_ridge()
    elif(method == 'CVElasticNet'):
        options = create_options_cv_elastic_net()
    elif(method == 'BestSubset'):
        options = create_options_best_subset()
    elif(method == 'RelaxedLasso'):
        options = create_options_relaxed_lasso()
    else:
        print('Bad Method Specification for Train')
        return
    options['returnModel'] = True
    options['printLoadings'] = False
    options['date'] = 'DataDate'
    for key in methodDict:
        options[key] = methodDict[key]
    return options



def print_timeperiod(data, dependentVar, options):
    '''print_timeperiod takes a a dependent varaible and a options dictionary, prints out the time period
    INPUTS:
        data: pandas df, df with the data
        dependentVar: string, name of dependent variable
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
    OUTPUTS:
        printed stuff
    '''
    print ('Dependent Variable is ' + dependentVar)
    if(options['time_period'] == 'all'):
        sortedValues = data.sort_values(options['date'])[options['date']].reset_index(drop=True)
        n = sortedValues.shape[0]
        beginDate = sortedValues[0]
        endDate = sortedValues[n-1]
        print ('Time period is between ' + num_to_month(beginDate.month) +  ' ' + str(beginDate.year) + ' to ' + num_to_month(endDate.month) +  ' ' + str(endDate.year) + ' inclusive   ')
    else:
        print ('Time period is ' + options['timeperiod'])

def display_factor_loadings(intercept, coefs, factorNames, options):
    '''display_factor_loadings takes an intercept, coefs, factorNames and options dict, and prints the factor loadings in a readable way
    INPUTS:
        intercept: float, intercept value
        coefs: np array, coeficients from pandas df
        factorNames: list, names of the factors
        options: dict, should contain at least one key, nameOfReg
            nameOfReg: string, name for the regression
    Outputs:
        output is printed
    '''
    loadings = np.insert(coefs, 0, intercept)
    if('name_of_reg' not in options.keys()):
        name = 'No Name'
    else:
        name = options['name_of_reg']
    out = pd.DataFrame(loadings, columns=[name])
    out = out.transpose()
    fullNames = ['Intercept'] + factorNames
    out.columns = fullNames
    print(out)

def best_subset(x,y,l_0):
    # Mixed Integer Programming in feature selection
    M = 1000
    n_factor = x.shape[1]
    z = cp.Variable(n_factor, boolean=True)
    beta = cp.Variable(n_factor)
    alpha = cp.Variable(1)

    def MIP_obj(x,y,b,a):
        return cp.norm(y-cp.matmul(x,b)-a,2)

    best_subset_prob = cp.Problem(cp.Minimize(MIP_obj(x, y, beta, alpha)),
                             [cp.sum(z)<=l_0, beta+M*z>=0, M*z>=beta])
    best_subset_prob.solve(solver='ECOS_BB')
    return alpha.value, beta.value


#First function, linear factor model build
def linear_regression(data, dependentVar, factorNames, options):
    '''linear_regression takes in a dataset and returns the factor loadings using least squares regression
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    #first filter down to the time period
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #perform linear regression
    linReg = LinearRegression(fit_intercept=True)
    linReg.fit(newData[factorNames], newData[dependentVar])

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        # Now print the factor loadings
        display_factor_loadings(linReg.intercept_, linReg.coef_, factorNames, options)

    if(options['return_model']):
        return linReg


def lasso_regression(data, dependentVar, factorNames, options):
    '''lasso_regression takes in a dataset and returns the factor loadings using lasso regression
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            printLoadings: boolean, if true, prints the coeficients

            date: name of datecol
            returnModel: boolean, if true, returns model
            alpha: float, alpha value for LASSO regression
            NOTE: SKLearn calles Lambda Alpha.  Also, it uses a scaled version of LASSO argument, so here I scale when converting lambda to alpha
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    if('lambda_hat' not in options.keys()):
        print ('lambda_hat not specified in options')
        return

    #first filter down to the time period
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #perform linear regression
    lassoReg = Lasso(alpha=options['lambda_hat'], fit_intercept=True)
    lassoReg.fit(newData[factorNames], newData[dependentVar])

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('lambda_hat = ' + str(options['lambda_hat']))

        #Now print the factor loadings
        display_factor_loadings(lassoReg.intercept_, lassoReg.coef_, factorNames, options)

    if(options['return_model']):
        return lassoReg

def ridge_regression(data, dependentVar, factorNames, options):
    '''ridge_regression takes in a dataset and returns the factor loadings using ridge regression
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
            lambda: float, alpha value for Ridge regression
            printLoadings: boolean, if true, prints the coeficients
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    if('lambda' not in options.keys()):
        print ('lambda not specified in options')
        return

    #first filter down to the time period
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #perform linear regression
    ridgeReg = Ridge(alpha=options['lambda'], fit_intercept=True)
    ridgeReg.fit(newData[factorNames], newData[dependentVar])

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('lambda = ' + str(options['lambda']))

        #Now print the factor loadings
        display_factor_loadings(ridgeReg.intercept_, ridgeReg.coef_, factorNames, options)

    if(options['return_model']):
        return ridgeReg

def best_subset_regression(data, dependentVar, factorNames, options):
    '''best_subset_regression takes in a dataset and returns the factor loadings using best subset regression
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
            maxVars: int, maximum number of factors that can have a non zero loading in the resulting regression
            printLoadings: boolean, if true, prints the coeficients
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    #Check dictionary for maxVars option
    if('max_vars' not in options.keys()):
        print ('max_vars not specified in options')
        return

    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #perform linear regression
    alpha, beta = best_subset(data[factorNames].values, data[dependentVar].values, options['max_vars'])
    #round beta values to zero
    beta[np.abs(beta) <= 1e-7] = 0.0

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('Max Number of Non-Zero Variables is ' + str(options['max_vars']))

        #Now print the factor loadings
        display_factor_loadings(alpha, beta, factorNames, options)

    if(options['return_model']):
        out = LinearRegression()
        out.intercept_ = alpha[0]
        out.coef_ = beta
        return out

def cross_validated_lasso_regression(data, dependentVar, factorNames, options):
    '''cross_validated_lasso_regression takes in a dataset and returns the factor loadings using lasso regression and cross validating the choice of lambda
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
            printLoadings: boolean, if true, prints the coeficients

            maxLambda: float, max lambda value passed
            nLambdas: int, number of lambda values to try
            randomState: integer, sets random state seed
            nFolds: number of folds
            NOTE: SKLearn calles Lambda Alpha.  Also, it uses a scaled version of LASSO argument, so here I scale when converting lambda to alpha
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    #Test timeperiod
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #Do CV Lasso
    alphas = np.logspace(-12, np.log(options['max_lambda_hat']), base=np.exp(1), num=options['n_lambda_hat'])
    #alphas = np.linspace(1e-12, alphaMax, options['nAlphas'])
    if(options['random_state'] == 'none'):
        lassoTest = Lasso(fit_intercept=True)
    else:
        lassoTest = Lasso(random_state = options['random_state'], fit_intercept=True)

    tuned_parameters = [{'alpha': alphas}]

    clf = GridSearchCV(lassoTest, tuned_parameters, cv=options['n_folds'], refit=True)
    clf.fit(newData[factorNames],newData[dependentVar])
    lassoBest = clf.best_estimator_
    alphaBest = clf.best_params_['alpha']

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('Best lambda_hat = ' + str(alphaBest))
        #Now print the factor loadings
        display_factor_loadings(lassoBest.intercept_, lassoBest.coef_, factorNames, options)

    if(options['return_model']):
        return lassoBest

def cross_validated_ridge_regression(data, dependentVar, factorNames, options):
    '''cross_validated_ridge_regression takes in a dataset and returns the factor loadings using ridge regression and choosing lambda via ridge regression
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
            printLoadings: boolean, if true, prints the coeficients

            maxLambda: float, max lambda value passed
            nLambdas: int, number of lambda values to try
            randomState: integer, sets random state seed
            nFolds: number of folds
            NOTE: SKLearn calles Lambda Alpha.  So I change Lambda -> Alpha in the following code
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    #Test timeperiod
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #Do CV Lasso
    alphaMax = options['max_lambda']
    alphas = np.logspace(-12, np.log(alphaMax), num=options['n_lambda'], base=np.exp(1))
    if(options['randomState'] == 'none'):
        ridgeTest = Ridge(fit_intercept=True)
    else:
        ridgeTest = Ridge(random_state = options['randomState'], fit_intercept=True)

    tuned_parameters = [{'alpha': alphas}]

    clf = GridSearchCV(ridgeTest, tuned_parameters, cv=options['n_folds'], refit=True)
    clf.fit(newData[factorNames],newData[dependentVar])
    ridgeBest = clf.best_estimator_
    alphaBest = clf.best_params_['alpha']

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('Best Lambda = ' + str(alphaBest))
        #Now print the factor loadings
        display_factor_loadings(ridgeBest.intercept_, ridgeBest.coef_, factorNames, options)

    if(options['return_model']):
        return ridgeBest

def cross_validated_elastic_net_regression(data, dependentVar, factorNames, options):
    '''cross_validated_elastic_net_regression takes in a dataset and returns the factor loadings using elastic net, also chooses alpha and l1 ratio via cross validation
    INPUTS:
        data: pandas df, data matrix, should constain the date column and all of the factorNames columns
        dependentVar: string, name of dependent variable
        factorNames: list, elements should be strings, names of the independent variables
        options: dictionary, should constain at least two elements, timeperiod, and date
            timeperiod: string, if == all, means use entire dataframe, otherwise filter the df on this value
            date: name of datecol
            returnModel: boolean, if true, returns model
            printLoadings: boolean, if true, prints the coeficients

            maxLambda: float, max lambda value passed
            nLambdas: int, number of lambda values to try
            maxL1Ratio: float
            randomState: integer, sets random state seed
            nFolds: number of folds
            NOTE: SKLearn calles Lambda Alpha.  So I change Lambda -> Alpha in the following code
    Outputs:
        reg: regression object from sikitlearn
        also prints what was desired
    '''
    #Test timeperiod
    if(options['time_period'] == 'all'):
        newData = data.copy()
    else:
        newData = data.copy()
        newData = newData.query(options['time_period'])

    #Do CV Lasso
    alphaMax = options['max_lambda_hat']
    alphas = np.logspace(-12, np.log(alphaMax), num=options['n_lambda_hat'])
    l1RatioMax = options['max_l1_ratio']
    l1Ratios = np.linspace(1e-6, l1RatioMax, options['n_l1_ratio'])
    if(options['random_state'] == 'none'):
        elasticNetTest = ElasticNet(fit_intercept=True)
    else:
        elasticNetTest = ElasticNet(random_state = options['random_state'], fit_intercept=True)

    tuned_parameters = [{'alpha': alphas, 'l1_ratio': l1Ratios}]

    clf = GridSearchCV(elasticNetTest, tuned_parameters, cv=options['n_folds'], refit=True)
    clf.fit(newData[factorNames],newData[dependentVar])
    elasticNetBest = clf.best_estimator_
    alphaBest = clf.best_params_['alpha']
    l1RatioBest = clf.best_params_['l1_ratio']

    if (options['print_loadings'] == True):
        #Now print the results
        print_timeperiod(newData, dependentVar, options)
        print('Best lambda_hat = ' + str(alphaBest))
        print('Best l1 ratio = ' + str(l1RatioBest))
        #Now print the factor loadings
        display_factor_loadings(elasticNetBest.intercept_, elasticNetBest.coef_, factorNames, options)

    if(options['return_model']):
        return elasticNetBest


def run_factor_model(data, dependentVar, factorNames, method, options):
    '''run_Factor_model allows you to specify the method to create a model, returns a model object according to the method you chose
    INPUTS:
        data: pandas df, must contain the columns specified in factorNames and dependentVar
        dependentVar: string, dependent variable
        factorNames: list of strings, names of independent variables
        method: string, name of method to be used.  Supports OLS, LASSO, CVLASSO atm
        options: dictionary object, controls the hyperparameters of the method
    Outputs:
        out: model object'''

    #Make sure the options dictionary has the correct settings
    options['return_model'] = True
    options['print_loadings'] = False

    #Now create the appropriate model
    if (method == 'OLS'): #run linear model
        return linear_regression(data, dependentVar, factorNames, options)
    if (method == 'LASSO'):
        return lasso_regression(data, dependentVar, factorNames, options)
    if (method == 'Ridge'):
        return ridge_regression(data, dependentVar, factorNames, options)
    if (method == 'CVLasso'):
        return cross_validated_lasso_regression(data, dependentVar, factorNames, options)
    if (method == 'CVRidge'):
        return cross_validated_ridge_regression(data, dependentVar, factorNames, options)
    if (method == 'CVElasticNet'):
        return cross_validated_elastic_net_regression(data, dependentVar, factorNames, options)
    if (method == 'BestSubset'):
        return best_subset_regression(data, dependentVar, factorNames, options)
    else:
        print ('Method ' + method + ' not supported')

# Function to create a time series of factor loadings using a trailing window
def compute_trailing_factor_regressions(data, dependentVar, factorNames, window, method, options, dateCol='Date', printTime=False):
    '''compute_trailing_factor_regressions computes the factor regresssions using a trailing window, returns a pandas df object
    INPUTS:
        data: pandas df, must constain the columns dependentVar, and the set of columns factorNames
        dependentVar: string, names the dependent variable, must be a column in the dataframe data
        factorNames: list of string, elements must be members
        window: int, lookback window, measured in number of trading days
        method: string, can be OLS, LASSO or CVLasso
        options: dictionary, options dictionary
        dateCol: string, name of date column, also must be included in data
        printTime: boolean, if True, prints time it took to run the regressions
    Outputs:
        regressionValues: pandas df, rows should be different dates, columns should be factor loadings calculated using the trailing window
    '''
    if(printTime):
        start = time.time()
    options['return_model'] = True
    options['print_loadings'] = False
    days = list(np.sort(data[dateCol].unique()))
    listOfFactorsAndDate = [dateCol] + factorNames
    regressionValues = pd.DataFrame(columns=listOfFactorsAndDate)
    for i in range(window, len(days)):
        #Filter the data
        filtered = data[(data[dateCol] <= days[i]) & (data[dateCol] >= days[i-window])]
        #Run the regression
        reg = run_factor_model(filtered, dependentVar, factorNames, method, options)
        #Append the regression values
        newRow = pd.DataFrame(reg.coef_)
        newRow = newRow.transpose()
        newRow.columns = factorNames
        newRow[dateCol] = days[i]
        regressionValues = regressionValues.append(newRow, sort=True)
    if(printTime):
        print('regression took ' + str((time.time() - start)/60.) + ' minutes')
    return regressionValues

#Asorted Nonsense
def num_to_month(month):
    #num to month returns the name of the month, input is an integer
    if (month==1):
        return 'January'
    if (month==2):
        return 'Febuary'
    if (month==3):
        return 'March'
    if (month==4):
        return 'April'
    if (month==5):
        return 'May'
    if (month==6):
        return 'June'
    if (month==7):
        return 'July'
    if (month==8):
        return 'August'
    if (month==9):
        return 'September'
    if (month==10):
        return 'October'
    if (month==11):
        return 'November'
    if (month==12):
        return 'December'


def data_time_periods(data, dateName):
    '''data_time_periods figures out if the data is daily, weekly, monthly, etc
    INPUTS:
        data: pandas df, has a date column in it with column name dateName
        dateName: string, name of column to be analysed
    '''
    secondToLast = data[dateName].tail(2)[:-1]
    last = data[dateName].tail(1)
    thingy = (last.values - secondToLast.values).astype('timedelta64[D]') / np.timedelta64(1, 'D')
    thingy = thingy[0]
    if (thingy > 200):
        return 'yearly'
    elif(thingy > 20):
        return 'monthly'
    elif(thingy > 5):
        return 'weekly'
    else:
        return 'daily'

In [4]:
#@title Package Uploads

#import all the necessary packages
import numpy as np #for numerical array data
import pandas as pd #for tabular data
import matplotlib.pyplot as plt #for plotting purposes

from sklearn.metrics import r2_score

%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import importlib as imp
import warnings
warnings.filterwarnings('ignore')

Read data

In [5]:
all_data = pd.read_csv(dataPath)
all_data[dateName] = pd.to_datetime(all_data[dateName])
all_data = all_data[all_data['Date'] <= '2014-12-01'].copy()

In [6]:
all_data.head()

,Date,World Equities,10-year US Treasuries,High Yield,Inflation Protection,Currency Protection,U.S. Equity,SP500 Total Return,S&P 500,International Equity,U.S. Treasury 20 years,Corporate Bond,Real Estate,Commodity,TIPS
0,1995-01-01,-0.020349,0.022922,0.032048,0.006524,-0.003404,0.022239,0.025954,0.025897,-0.039228,0.025987,0.017042,-0.014847,-0.023908,0.029446
1,1995-02-01,0.010680,0.030014,0.013531,-0.038375,-0.020587,0.036258,0.035540,0.038974,-0.000434,0.026960,0.029870,0.032460,0.002221,-0.008361
2,1995-03-01,0.045804,0.007195,0.025871,-0.023184,-0.042060,0.030791,0.035454,0.029502,0.061949,0.009477,0.011937,-0.005961,0.019558,-0.015989
3,1995-04-01,0.036372,0.016973,0.030836,-0.003600,-0.015827,0.022403,0.026256,0.029414,0.037154,0.017695,0.010995,0.002259,0.038085,0.013373
4,1995-05-01,0.011040,0.054900,0.006743,-0.054282,0.009121,0.036521,0.040894,0.039949,-0.009769,0.076715,0.053217,0.045025,-0.009417,0.000618


When estimating more than 3 parameters, OLS regression leads to larger mean squared error than other methods. In short, we will introduce a penalty as a way to decrease overall error.

We will use LASSO Regression, ElasticNet (combo of LASSO & Ridge Regression), and "best subset selection" 

We define LASSO regression as the following optimization problem. (**same** as OLS but with penalty term)

As before, n is the number of data points, and m is the number factors.

\begin{equation*}
    {\hat{\beta}}^{\text{LASSO}} = argmin_{\beta}\bigg\{\sum_{t=1}^{n} (y_t - X_t^T \beta)^2 + \lambda \sum_{j=1}^{m}|\beta_j| \bigg\}
\end{equation*}

\begin{equation*}
    {\frac{\lambda}{2*n} = \hat{\lambda}}
\end{equation*}

We use L1 Norm penalty, this is just to clear up notation

The following codeblock calculates the LASSO model with $\hat{\lambda}$ =.00001

In [7]:
train = all_data[(all_data['Date'] <= '2012-12-01') & (all_data['Date'] >= '1997-03-01')].copy()
test = all_data[all_data['Date'] > '2012-12-01'].copy()

In [8]:
options = create_options_lasso()
options['lambda_hat'] = .00001 #small lambda
options['print_loadings'] = True
options['name_of_reg'] = 'LASSO Regression with small Lambda'
lasso_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
lambda_hat = 1e-05
                                    Intercept  World Equities  \
LASSO Regression with small Lambda   0.000049        0.989737   

                                    10-year US Treasuries  High Yield  \
LASSO Regression with small Lambda               0.063311         0.0   

                                    Inflation Protection  Currency Protection  
LASSO Regression with small Lambda                   0.0             0.308432  


Larger lambda = more aggresive = more shrinkage

In [9]:
options = create_options_lasso()
options['lambda_hat'] = .001 #The input alpha value
options['print_loadings'] = True
options['name_of_reg'] = 'LASSO Reg with large Lambda'
lasso_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
lambda_hat = 0.001
                             Intercept  World Equities  10-year US Treasuries  \
LASSO Reg with large Lambda   0.002656        0.550637                   -0.0   

                             High Yield  Inflation Protection  \
LASSO Reg with large Lambda         0.0                   0.0   

                             Currency Protection  
LASSO Reg with large Lambda                 -0.0  


Use cross-validation to find appropriate lambda value (5 folds)

In [10]:
options = create_options_cv_lasso() #make sure that you are doing cv_lasso when performing cross validation with lasso
options['name_of_reg'] = 'CV Lasso'
options['max_lambda_hat'] = .001 #This specifies the maximum Alpha value tested by cross validation, minimum value is zero
options['return_model'] = True
options['n_folds'] = 5 #This states the number of folds
lasso_model_train = cross_validated_lasso_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
Best lambda_hat = 1.6667957547420788e-05
          Intercept  World Equities  10-year US Treasuries  High Yield  \
CV Lasso   0.000242        0.976867               0.036007         0.0   

          Inflation Protection  Currency Protection  
CV Lasso                  -0.0              0.27196  


best lambda: .0000166

## Using ElasticNet

\begin{equation*}
    {\hat{\beta}}^{\text{LASSO}} = argmin_{\beta}\bigg\{\sum_{t=1}^{n} (y_t - X_t^T \beta)^2 + \lambda_1 \sum_{j=1}^{m}|\beta_j| \bigg\}
\end{equation*}

\begin{equation*}
    {\hat{\beta}}^{\text{Ridge}} = argmin_{\beta}\bigg\{\sum_{t=1}^{n} (y_t - X_t^T \beta)^2 + \lambda_2||\beta||_2^2 \bigg\}
\end{equation*}

\begin{equation*}
    {\hat{\beta}}^{\text{Elastic Net}} = argmin_{\beta}\bigg\{\sum_{t=1}^{n} (y_t - X_t^T \beta)^2 + \lambda_1 \sum_{j=1}^{m}|\beta_j| + \lambda_2||\beta||_2^2 \bigg\}
\end{equation*}


Scikit-learn's ElasticNet:

\begin{equation*}
    \hat{\beta}^{\text{Elastic Net}} = 
    \underset{\beta}{\arg\min} \left\{
        \sum_{t=1}^{n} (y_t - X_t^\top \beta)^2 
        + \lambda \cdot \text{l1\_ratio} \sum_{j=1}^{m} |\beta_j| 
        + \frac{1}{2} \lambda \cdot (1 - \text{l1\_ratio}) \|\beta\|_2^2
    \right\}
\end{equation*}


\begin{equation*}
    \hat{\beta}^{\text{OLS}} = \underset{\beta}{\arg\min} \sum_{t=1}^{n} (y_t - X_t^\top \beta)^2
\end{equation*}

- y_t: Actual response value for observation t
- X_t^\top \beta: Predicted value for observation t using covariates X_t and coefficients \beta
- The expression (y_t - X_t^\top \beta)^2 measures the squared error of the prediction
- We sum over all n observations: \sum_{t=1}^{n}
- The goal is to minimize this sum → find \beta that makes total prediction error as small as possible
- \hat{\beta}^{\text{OLS}} is the optimal set of coefficients that achieves this

In [11]:
options = create_options_cv_elastic_net()
options['name_of_reg'] = 'CV Elastic Net'
options['max_lambda_hat'] = .01
options['n_folds'] = 5
options['return_model'] = True
el_model_train = cross_validated_elastic_net_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
Best lambda_hat = 1.759702749929848e-05
Best l1 ratio = 0.9378947894736842
                Intercept  World Equities  10-year US Treasuries  High Yield  \
CV Elastic Net   0.000245        0.976261               0.035807         0.0   

                Inflation Protection  Currency Protection  
CV Elastic Net                  -0.0             0.270812  


Most of the penalty is coming from the Ridge, a little from the Lasso

## Best Subset Regression

\begin{equation*}
    {\hat{\beta}}^{\text{Best Subset}} = argmin_{\beta}\bigg\{\sum_{t=1}^{n} (y_t - X_t^T \beta)^2 \bigg\}.
\end{equation*}

\begin{equation*}
\begin{aligned}
    & \sum_{i=1}^{\text{max\_vars}} z_i \leq \text{max\_vars} \\
    & Mz + \beta \geq 0 \\
    & \beta \leq Mz \\
    & \textbf{z} \text{ is binary}
\end{aligned}
\end{equation*}



maxVars = 2

In [12]:
import ecos

options = create_options_best_subset()
options['max_vars'] = 2
options['name_of_reg'] = 'Best Subset with maxVars = 2'
best_subset_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
Max Number of Non-Zero Variables is 2
                              Intercept  World Equities  \
Best Subset with maxVars = 2    0.00041          0.9917   

                              10-year US Treasuries  High Yield  \
Best Subset with maxVars = 2                    0.0         0.0   

                              Inflation Protection  Currency Protection  
Best Subset with maxVars = 2                   0.0             0.324069  


3 vars

In [13]:
options = create_options_best_subset()
options['max_vars'] = 3
options['return_model'] = True
options['name_of_reg'] = 'Best Subset with maxVars = 3'
best_subset_3 = best_subset_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
Max Number of Non-Zero Variables is 3
                              Intercept  World Equities  \
Best Subset with maxVars = 3   -0.00024        1.009086   

                              10-year US Treasuries  High Yield  \
Best Subset with maxVars = 3               0.104319         0.0   

                              Inflation Protection  Currency Protection  
Best Subset with maxVars = 3                   0.0             0.363219  


Assumptions:

 * Global Equities: 8%
 * 10-Year Treasury Bonds: 2%
 * High Yield Bonds: 5%
 * Inflation Protection: -.3%
 * Currency Protection: 0%

In [14]:
#@title Loading the OLS model from last lab

options = create_options()
options['name_of_reg'] = 'OLS full data'
options['return_model'] = True
ols_model_train = linear_regression(train, 'SP500 Total Return', factorName, options)

Dependent Variable is SP500 Total Return
Time period is between March 1997 to December 2012 inclusive   
               Intercept  World Equities  10-year US Treasuries  High Yield  \
OLS full data  -0.000384        1.006532                0.13231   -0.000067   

               Inflation Protection  Currency Protection  
OLS full data               0.05813              0.37356  


In [15]:
models = [ols_model_train, lasso_model_train, el_model_train, best_subset_3]

comparison = pd.DataFrame(np.zeros((4, 7)), columns=['Intercept'] + factorName +
                          ['Implied Expected Return S&P 500 (Annual)'])
expected_returns = pd.DataFrame(np.array([[.08/12, .02/12, .05/12, -.003/12, 0/12]]), columns=factorName)

for i in range(len(models)):
    model = models[i]
    comparison.loc[i,'Intercept'] = model.intercept_
    comparison.loc[i, factorName] = model.coef_
    comparison.loc[i, 'Implied Expected Return S&P 500 (Annual)'] = 12*model.predict(expected_returns)

comparison.index = ['OLS', 'LASSO', 'Elastic Net', 'Best Subset']

Seeing comparison with all of our models

In [16]:
comparison

,Intercept,World Equities,10-year US Treasuries,High Yield,Inflation Protection,Currency Protection,Implied Expected Return S&P 500 (Annual)
OLS,-0.000384,1.006532,0.132310,-0.000067,0.05813,0.373560,0.078384
LASSO,0.000242,0.976867,0.036007,0.000000,-0.00000,0.271960,0.081768
Elastic Net,0.000245,0.976261,0.035807,0.000000,-0.00000,0.270812,0.081758
Best Subset,-0.000240,1.009086,0.104319,0.000000,0.00000,0.363219,0.079933


Using R2_Score from sklearn to calculate error on our testing set from our model.

In [17]:
models = [ols_model_train, lasso_model_train, el_model_train, best_subset_3]
predictions = pd.DataFrame(np.zeros((4,1)), columns=['R^2 on Testing Set'])

for i in range(len(models)):
    model = models[i]
    predictions.loc[i, 'R^2 on Testing Set'] = r2_score(test['SP500 Total Return'],
                                                        model.predict(test[factorName]))
predictions.index = ['OLS', 'LASSO', 'Elastic Net', 'Best Subset']

In [18]:
predictions

,R^2 on Testing Set
OLS,0.855772
LASSO,0.858052
Elastic Net,0.857804
Best Subset,0.862486
